# Getting Data from Internet

In [40]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

4396.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


--2024-02-27 17:22:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
正在解析主机 raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
正在连接 raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：1115394 (1.1M) [text/plain]
正在保存至: “input.txt.1”

input.txt.1         100%[===================>]   1.06M  4.35MB/s  用时 0.2s      

2024-02-27 17:22:08 (4.35 MB/s) - 已保存 “input.txt.1” [1115394/1115394])



In [41]:
# read it and inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [42]:
print("length of the dataset in characters: ", len(text))

length of the dataset in characters:  1115394


In [43]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



To see individual character of the whole text

In [44]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


# Encoder and Decoder

## stoi and encoder
stoi 是一个字典，它将字符映射到整数。字典的名称 stoi 是 "string to integer" 的缩写。这个字典是通过字典推导式创建的，字典推导式的输入是 enumerate(chars) 的结果。enumerate(chars) 会为 chars 列表中的每个元素生成一个索引，然后我们将字符作为键，索引作为值，存储在 stoi 字典中。

例如，如果 chars 是 ['a', 'b', 'c']，那么 stoi 就会是 {'a': 0, 'b': 1, 'c': 2}。

encode 是一个函数，它接受一个字符串 s，并返回一个整数列表。这个函数是通过 lambda 表达式定义的，lambda 表达式是一种创建小型匿名函数的方式。encode 函数的工作方式是，对于输入字符串 s 中的每个字符 c，它都会查找 stoi 字典中 c 对应的值（即 stoi[c]），然后将这些值收集到一个列表中。

## itos and decoder
itos 是一个字典，它将整数映射到字符。字典的名称 itos 是 "integer to string" 的缩写。这个字典是通过字典推导式创建的，字典推导式的输入是 enumerate(chars) 的结果。enumerate(chars) 会为 chars 列表中的每个元素生成一个索引，然后我们将索引作为键，字符作为值，存储在 itos 字典中。

例如，如果 chars 是 ['a', 'b', 'c']，那么 itos 就会是 {0: 'a', 1: 'b', 2: 'c'}。

decode 是一个函数，它接受一个整数列表 l，并返回一个字符串。这个函数是通过 lambda 表达式定义的，lambda 表达式是一种创建小型匿名函数的方式。decode 函数的工作方式是，对于输入列表 l 中的每个整数 i，它都会查找 itos 字典中 i 对应的值（即 itos[i]），然后将这些值收集到一个字符串中。

例如，如果 l 是 [0, 1, 2]，那么 decode(l) 就会是 "abc"（假设 itos 是上面的例子中的字典）。


In [45]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello NanoGPT"))
print(decode(encode("Hello NanoGPT")))

[20, 43, 50, 50, 53, 1, 26, 39, 52, 53, 19, 28, 32]
Hello NanoGPT


## 使用 Torch 进行包装

In [46]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

# Training Data

In [47]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

在 LLM 训练的时候，一般不会将整个数据库都拿来训练 Transformer，而是会将数据“分块”，一块一块地进行训练

In [48]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

在训练的时候，我们希望 Transformer 能拥有 “预测下一个词“ 的能力。

比如说上述的序列中：
1. 给出一个 ”18“ 的词，Transformer 能够预测下一个词是 ”47“
2. 给出一个序列为 ”18，47“，Transformer 能够预测下一个词是 ”56“
依次类推。

关联信息：[waytoagi.feishu.cn](https://waytoagi.feishu.cn/wiki/LnMOwmKIwihcRakRalcczNgrnjf)

In [49]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


上述的是一个 block 进行训练，真实场景下，我们通常是 “一个批次” 进行训练，即将 “多个 block 合为一个 batch” 进行训练。

以下代码将引入 batch 的维度：

In [50]:
torch.manual_seed(114514) # :)

batch_size = 4  # 定义有多少个batch并行训练     how many independent sequences will we process in parallel?
block_size = 8  # 每一个block的最大长度        what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))   # batch_size 指定生成的长度
    #print('ix: ', str(ix))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----------------')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[57, 43, 50, 63,  1, 39, 58,  1],
        [46,  1, 61, 39, 57, 46,  5, 42],
        [58,  1, 54, 56, 43, 54, 39, 56],
        [47, 52, 49,  6,  1, 53, 56,  1]])
targets:
torch.Size([4, 8])
tensor([[43, 50, 63,  1, 39, 58,  1, 51],
        [ 1, 61, 39, 57, 46,  5, 42,  1],
        [ 1, 54, 56, 43, 54, 39, 56, 43],
        [52, 49,  6,  1, 53, 56,  1, 40]])
----------------
when input is [57] the target: 43
when input is [57, 43] the target: 50
when input is [57, 43, 50] the target: 63
when input is [57, 43, 50, 63] the target: 1
when input is [57, 43, 50, 63, 1] the target: 39
when input is [57, 43, 50, 63, 1, 39] the target: 58
when input is [57, 43, 50, 63, 1, 39, 58] the target: 1
when input is [57, 43, 50, 63, 1, 39, 58, 1] the target: 51
when input is [46] the target: 1
when input is [46, 1] the target: 61
when input is [46, 1, 61] the target: 39
when input is [46, 1, 61, 39] the target: 57
when input is [46, 1, 61, 39, 57] the target: 46
when inp

我们输入给 Transformer 训练的是这样的数据，每一行都是一个训练样本：
```py
tensor([[43, 50, 63,  1, 39, 58,  1, 51],
        [ 1, 61, 39, 57, 46,  5, 42,  1],
        [ 1, 54, 56, 43, 54, 39, 56, 43],
        [52, 49,  6,  1, 53, 56,  1, 40]])
```


In [51]:
print(xb) # our input to the transformer

tensor([[57, 43, 50, 63,  1, 39, 58,  1],
        [46,  1, 61, 39, 57, 46,  5, 42],
        [58,  1, 54, 56, 43, 54, 39, 56],
        [47, 52, 49,  6,  1, 53, 56,  1]])


# Bigram Language Model: The Simplest LLM Model

## 知识科普：交叉熵cross_entropy

交叉熵（Cross Entropy）是一种在机器学习和信息论中常用的损失函数，特别是在分类问题中。它用于衡量模型预测的概率分布与真实的概率分布之间的差异。

假设我们有一个真实的概率分布 P 和一个模型预测的概率分布 Q，那么 P 和 Q 之间的交叉熵定义为：`H(P, Q) = - Σ P(x) log(Q(x))`

其中，Σ 表示对所有可能的事件 x 的求和。

在分类问题中，真实的概率分布 P 通常是一个 one-hot 编码的向量，也就是说，对于正确的类别，其概率为 1，对于其他类别，其概率为 0。因此，交叉熵损失函数就变成了对于正确类别的预测概率的负对数似然。

交叉熵损失函数有一个很好的性质，那就是当模型的预测概率分布接近真实的概率分布时，交叉熵的值会很小，反之则会很大。因此，通过最小化交叉熵损失函数，我们可以让模型的预测概率分布尽可能接近真实的概率分布。

> 在以下神经网络中，我们使用 cross_entropy 作为计算 loss 值的方法

In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(114514)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
            # 根据pytorch的文档，cross_entropy的入参(minibatch,C,d1,....dk)，第二个维度是C，所以需要对源 logits 进行改造
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.5480, grad_fn=<NllLossBackward0>)

tCK!?.QVDheWEVuNLsj.VwhEgFldy&FueufK.Sl?n3PtXmZvsckrytSgrhgUGl KLepkJ!xSBv,RbXm.g !'JSscvB&NwPDXPJaY


在没有训练的时候，输出的内容都是乱码。

generate 第一个入参是 ‘zero’，在原始文本中表示”空格“，第二个 max_new_tokens 则是让模型推测文本的最大长度

接下来我们将开始做一些简答的训练：

# Traning the Transformer Model

先创建一个 PyTorch 的优化器，用于更新模型的参数以最小化损失函数。

优化器的类型是 AdamW，这是一种常用的优化算法，它结合了 Adam（自适应矩估计）算法的优点和权重衰减（也称为 L2 正则化）的优点。AdamW 优化器在训练深度学习模型时，特别是在处理大规模数据集和复杂模型时，通常可以得到更好的结果。

m.parameters() 是一个生成器，它返回模型 m 的所有可训练参数。这些参数是需要优化的对象。

lr=1e-3 设置了学习率为 0.001。学习率是一个超参数，它决定了模型参数在每次更新时的移动步长。如果学习率太大，可能会导致模型在训练过程中震荡和不稳定；如果学习率太小，可能会导致模型训练速度过慢，甚至可能无法收敛。选择合适的学习率通常需要一些实验和经验。

In [53]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [60]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.4437882900238037


In [61]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


By hans s feaist whay why ooowoures IO:
Ance!
Whe atomagh by sl is
Nuneand to ndreat haurothan hair. sifen th hite sol ce my w-luseamenitomeyepr rishak he por ir?
ETheph or fut tory d sof t tha tand.
ave isathedo ty tot eangod, bomman gry lo id
Andsuthy hinlst betheve onor quigid, inoun haral, at sw'l itim ld, es ne nn y Me I ie t
Y:
RORO, my b, far ia tss geveclin!
An t d me woutl tl ow, CHennt aso prybe;
CUSobin faye, tweryoursouge nin:
Wigmears d m keand, t
EShak,
Th d t t dizefunoutyo we sh 


在上述的训练方式中，每一个 token 字符之间都没有相互关联，他们不会 'talking to each other'。

接下来，我们将尝试让 token “产生关联”，让训练更有效。

> 注意：bigram 模型的代码都整合为一个 bigram.py 文件了